<a href="https://colab.research.google.com/github/daniloc7/Colab_RedesNeurais/blob/main/F_IA1_ClassificadorBin%C3%A1rio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
# Perceptron Algorithm on the Sonar Dataset
from random import seed
from random import randrange
from csv import reader
import numpy as np

# Sensibilidade TP/(TP+FN) 
# Especificidade TN/(TN+FP)

actualTotal = list()
predictedTotal = list()
# TP = 0
# TN = 0
# FN = 0
# FP = 0

# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	#print(dataset_split)
	return dataset_split

# 

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		#realiza uma comparação do dado atual e predito
		if actual[i] == predicted[i]:
			correct += 1
	#print(len(actual))
	return correct / float(len(actual)) * 100.0



# def atribuirValores():

# 	for i in range(len(actualTotal)):
# 		if (predictedTotal[i] == 1 and actualTotal[i] == 1):
# 			TP += 1
# 		elif(predictedTotal[i] == 0 and actualTotal[i] == 0):
# 			TN += 1
# 		elif(predictedTotal[i] == 1 and actualTotal[i] == 0):
# 			FN += 1
# 		elif(predictedTotal[i] == 0 and actualTotal[i] == 1):
# 			FP += 1	

		# print("TP: %i" % TP)
		# print("TN: %i" % TN)
		# print("FP: %i" % FP)
		# print("FN: %i" % FN)

def calcularFuncao(funcao):
	TP = 0
	TN = 0
	FN = 0
	FP = 0
	for i in range(len(actualTotal)):
		#calcula TP
		if (predictedTotal[i] == 1 and actualTotal[i] == 1):
			TP += 1
		elif(predictedTotal[i] == 0 and actualTotal[i] == 0):
			TN += 1
		elif(predictedTotal[i] == 1 and actualTotal[i] == 0):
			FN += 1
		elif(predictedTotal[i] == 0 and actualTotal[i] == 1):
			FP += 1

	# print("TP: %i" % TP)
	# print("TN: %i" % TN)
	# print("FP: %i" % FP)
	# print("FN: %i" % FN)
 
	if(funcao == "acur"):
		return (100 * (TP + TN)/(TP+FP+TN+FN))
	
	if(funcao == "sens"):
		return (100 * TP/(TP+FN))
	
	if(funcao == "espec"):
		return (100 * (TN/(TN+FP)))

# Evaluate an algorithm using a cross validation split
# Aqui tem os valores reais e preditos.
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	#print(len(dataset))
	for fold in folds:
		train_set = list(folds)
		#print(len(train_set))
		train_set.remove(fold)
		train_set = sum(train_set, [])
		#print(sum(train_set, []))
		test_set = list()
		#print(len(test_set))
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		i = 0
		scores.append(accuracy)
		# print(type(actual))
		while(i != len(actual) ):
			actualTotal.append(actual[i])
			predictedTotal.append(predicted[i])
			i += 1
	# print(len(actualTotal))
	return scores

def chamaCalculo():
	train_set = list(dataset)
	

# Make a prediction with weights
def predict(row, weights):
	activation = weights[0]
	for i in range(len(row)-1):
		activation += weights[i + 1] * row[i]
	return 1.0 if activation >= 0.0 else 0.0

# Estimate Perceptron weights using stochastic gradient descent
def train_weights(train, l_rate, n_epoch):
	weights = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		for row in train:
			prediction = predict(row, weights)
			error = row[-1] - prediction
			weights[0] = weights[0] + l_rate * error
			for i in range(len(row)-1):
				weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
	return weights

# Perceptron Algorithm With Stochastic Gradient Descent
def perceptron(train, test, l_rate, n_epoch):
	predictions = list()
	weights = train_weights(train, l_rate, n_epoch)
	for row in test:
		prediction = predict(row, weights)
		predictions.append(prediction)
	return(predictions)

# Test the Perceptron algorithm on the sonar dataset
seed(1)
# load and prepare data
filename = 'sonar.all-data.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert string class to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
#print(VP)
n_folds = 3
l_rate = 0.01
n_epoch = 500
scores = evaluate_algorithm(dataset, perceptron, n_folds, l_rate, n_epoch)
# atribuirValores()
print("Acurácia: %.2f%%"  % calcularFuncao("acur"))
print("Sensibilidade: %.2f%%"  % calcularFuncao("sens"))
print("Especificidade: %.2f%%"  % calcularFuncao("espec"))
# print(len(dataset))
# print(len(predictedTotal))
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Acurácia: 72.95%
Sensibilidade: 72.04%
Especificidade: 73.68%
Scores: [76.81159420289855, 69.56521739130434, 72.46376811594203]
Mean Accuracy: 72.947%
